# Pre-reqs

In [ ]:
pip install requests

# Base64-Encode file
AltText.ai requires a file url or base64-encoded file for processing. All my files are local, so encoding it is

In [1]:
import base64

def encode_image_to_base64(image_path):
    """
    Reads a local image file and returns its Base64-encoded string.

    :param image_path: Path to the local image file
    :return: Base64-encoded string of the image
    """
    try:
        with open(image_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
        return encoded_string
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage:
# base64_string = encode_image_to_base64("path/to/your/image.jpg")
# print(base64_string)


# API Call to create-image (AtlText.ai)


In [11]:
import requests

def create_image(image_base64, tags):
    api_url = 'https://alttext.ai/api/v1/images'
    """
    Posts a Base64-encoded image to the given API URL with specified tags.

    :param image_base64: Base64-encoded string of the image
    :param tags: List of tags associated with the image
    :param api_url: URL of the API endpoint
    :return: Response from the API
    """
    payload = {
        "image": {
            "raw": image_base64,
            "tags": tags
        },
    }

    headers = {
        "X-API-Key": "api_key_here",
        "Content-Type": "application/json"
    }

    try:
        response = requests.post(api_url, json=payload, headers=headers)
        print(response)
        return response.json()
    except Exception as e:
        print(f"An error occurred while posting to the API: {e}")
        return None

# Create & save output to CSV file

In [13]:
import csv

# CSV file setup
fieldnames = ['file_name', 'alt_text']

def create_csv_file(csv_file_name):
    # Always initialize (wipe) the CSV file with headers
    with open(csv_file_name, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()

def save_output_to_csv(file_path, data):
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Filename", "Alt Text"])
        writer.writerows(data)

# Iterate the images found at a file path

For each file in the file path
1. Encode the file
2. POST to /create-image url: https://alttext.ai/apidocs#tag/Images/operation/create-image
3. Save output csv file (image name and seo alt text)

In [17]:
import os
import re

csv_file_name = "AltTextAI_SEO_Output.csv"
directory_path = '/Volumes/KKP.2024/Photos/StyledShoot/FoxCanyon.06.03/Finals/SEO.Web'
csv_file_path = directory_path + "/" + csv_file_name
wedding_keywords = ["wedding", "adventurous love story", "timeless", "memorable", "stress-free day", "joyful"]

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', s)]

create_csv_file(csv_file_path)

output_data = []

try:
    for root, _, files in os.walk(directory_path):
        for file in files:
            if file.lower().endswith('.jpg'):
                file_path = os.path.join(root, file)
                encoded_file_path = encode_image_to_base64(file_path)
                response = create_image(encoded_file_path, wedding_keywords)
                print(response)

                if response and 'alt_text' in response:
                    output_data.append((file, response['alt_text']))
                else:
                    print(f"Failed to get alt_text for {file}")
        
    # Sort the output data based on the filenames
    output_data.sort(key=lambda x: natural_sort_key(x[0]))

    # Save the sorted data to the CSV file
    save_output_to_csv(csv_file_path, output_data)

except Exception as e:
    print(f"An error occurred while iterating over the directory: {e}")
    
    

<Response [200]>
{'asset_id': '3b7c8ec5f846ae14a4037253f802c556', 'url': None, 'alt_text': 'A black-and-white photo of a park with lush grass, several trees, and a path. To the right, a bench surrounds a large tree. In the background, there is a giant Connect Four game setup. The scene is calm and shaded by the overhead foliage.', 'tags': ['wedding', 'adventurous love story', 'timeless', 'memorable', 'stress-free day', 'joyful'], 'metadata': {}, 'created_at': 1722633261, 'errors': {}, 'error_code': None}
Processed idaho-wedding-inspiration-along-snake-river-canyon-at-fox-canyon-vineyards-2.jpg: A black-and-white photo of a park with lush grass, several trees, and a path. To the right, a bench surrounds a large tree. In the background, there is a giant Connect Four game setup. The scene is calm and shaded by the overhead foliage.
<Response [200]>
{'asset_id': '972597816db61e6984f797942d0f85ae', 'url': None, 'alt_text': 'A river meanders through a green valley with scattered bushes and t

# Issues Found
- Encoding of special characters isn't working properly. Mainly apostrophe's
- image number out of order making it challenging to match to squarespace